The main purpose of this notebook is to demonstrate how much money could be made from cross-exchange cryptocurrency trades under the following (very large) assumptions:

- Capital controls do not restrict trades.
- 0 transaction fees.
- Trades occur instantaneously.
- No limit to the amount of capital that we have.

For each coin, at any time (we can resample later), calculate maximum price dislocation across any exchange. We will buy the lower one and sell the higher one at the maximum volume (which is the lesser of the two volumes). This will be the amount of money we could make at that exact time (second, two seconds, etc). Summing across all time in our dataset, this yields the total amount of money we could make for each asset.

Observe that this process will be simplest if our dataframe has the following format:
- The outer level represents the security (27 coins on different exchanges).
- The middle level represnts the feature (volume or vwap).
- The inner level represents the exchange (binance_futures, binance_spots, binanceus, okx).

Note that we also can only arbitrage futures vs. futures and spot vs. spot, since we have no information on the expireation date of the futures.

Analyzing price dislocations will be simplest and most efficient in the following dataframe format because we will be able perform all calculations vectorially.

Let $price_i$ be the minimum price of some arbitrary coin and let $price_j$ be the maximum price of the same arbirary coin at some time $t$
$$\mu = \min(\text{volume}_i, \text{volume}_j) * (\text{price}_j - \text{price}_i)$$

                  price_i        price_j          i                j              volume[i]     volume[j]      instance_profit 
        9:30        10              8         binanceus     binance_futures           2            5              
        9:31        12              8            okx        binance_futures           3            7

where $(i, j)$ are the same coin but on different exchanges. They represent different indices in the dataframe. Note that NaN values simply mean that the crypto coin was not tradeable on some exchange at that time.

In the library, we can achieve our goal by grouping the dataframe into smaller dataframes by $i, j$ pairs such that we can perform only vectorial operations. Afterwards, we can merge these dataframes in chronological order. Once our dataframe is in this format, we can calculate our total arbitrage profits as 
$$\mu_{total} = \sum_{t_0}^{t_f}\min(\text{volume}_i, \text{volume}_j) * (\text{price}_j - \text{price}_i)$$
where $t_i$ and $t_f$ represents the start time and end time, respectively.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from functools import reduce
import datetime
from typing import List, Optional 
import pyarrow as pa
import pyarrow.parquet as pq
import sys
sys.path.append("../library/")
import crossexchangearb as cxa

In [2]:
spots = pd.read_parquet("../data/spots.parquet")
futures = pd.read_parquet("../data/futures.parquet")

In [14]:
df_futures = cxa.swaplevels(futures)
df_spots = cxa.swaplevels(spots)
df_futures.head()

APE_USDT                          \
                                   volume                vwap       
                          binance_futures okx binance_futures okx   
timestamp                                                           
2019-09-08 17:57:00+00:00             NaN NaN             NaN NaN   
2019-09-08 17:58:00+00:00             NaN NaN             NaN NaN   
2019-09-08 17:59:00+00:00             NaN NaN             NaN NaN   
2019-09-08 18:00:00+00:00             NaN NaN             NaN NaN   
2019-09-08 18:01:00+00:00             NaN NaN             NaN NaN   

                                AVAX_USDT                          \
                                   volume                vwap       
                          binance_futures okx binance_futures okx   
timestamp                                                           
2019-09-08 17:57:00+00:00             NaN NaN             NaN NaN   
2019-09-08 17:58:00+00:00             NaN NaN             NaN NaN   
2019-09-08 17:59:00+00:00             NaN NaN             NaN NaN   
2019-09-08 18:00:00+00:00             NaN NaN             NaN NaN   
2019-09-08 18:01:00+00:00             NaN NaN             NaN NaN   

                                 AXS_USDT      ...       UNFI_USDT  \
                                   volume      ...          volume   
                          binance_futures okx  ... binance_futures   
timestamp                                      ...                   
2019-09-08 17:57:00+00:00             NaN NaN  ...             NaN   
2019-09-08 17:58:00+00:00             NaN NaN  ...             NaN   
2019-09-08 17:59:00+00:00             NaN NaN  ...             NaN   
2019-09-08 18:00:00+00:00             NaN NaN  ...             NaN   
2019-09-08 18:01:00+00:00             NaN NaN  ...             NaN   

                                               WAVES_USDT                      \
                                     vwap          volume                vwap   
                          binance_futures binance_futures okx binance_futures   
timestamp                                                                       
2019-09-08 17:57:00+00:00             NaN             NaN NaN             NaN   
2019-09-08 17:58:00+00:00             NaN             NaN NaN             NaN   
2019-09-08 17:59:00+00:00             NaN             NaN NaN             NaN   
2019-09-08 18:00:00+00:00             NaN             NaN NaN             NaN   
2019-09-08 18:01:00+00:00             NaN             NaN NaN             NaN   

                                     XRP_USDT                          
                                       volume                vwap      
                          okx binance_futures okx binance_futures okx  
timestamp                                                              
2019-09-08 17:57:00+00:00 NaN             NaN NaN             NaN NaN  
2019-09-08 17:58:00+00:00 NaN             NaN NaN             NaN NaN  
2019-09-08 17:59:00+00:00 NaN             NaN NaN             NaN NaN  
2019-09-08 18:00:00+00:00 NaN             NaN NaN             NaN NaN  
2019-09-08 18:01:00+00:00 NaN             NaN NaN             NaN NaN  

[5 rows x 90 columns]

In [21]:
cxa.arbitrage(df_futures, "AVAX_USDT")

,price_i,price_j,i,j,min_volume,instant_profit
timestamp,,,,,,
2021-01-01 15:02:00+00:00,3.695451,3.725500,binance_futures,okx,137.169200,4.121787
2021-01-01 15:03:00+00:00,3.695450,3.724216,binance_futures,okx,33.754300,0.970977
2021-01-01 15:04:00+00:00,3.695450,3.724216,binance_futures,okx,0.000000,0.000000
2021-01-01 15:05:00+00:00,3.695449,3.724216,binance_futures,okx,0.000000,0.000000
2021-01-01 15:06:00+00:00,3.695449,3.724216,binance_futures,okx,0.000000,0.000000
...,...,...,...,...,...,...
2023-01-31 13:15:00+00:00,35.391414,45.890622,binance_futures,okx,13.863603,145.556846
2023-01-31 13:16:00+00:00,35.391409,45.890617,binance_futures,okx,16.577327,174.048813
2023-01-31 13:17:00+00:00,35.391399,45.890617,binance_futures,okx,1.870275,19.636424


See results in the results directory in the repository. More analysis should be done in order to truly understand the extent of capital controls and analyze any patterns in the price dislocations.